<a href="https://colab.research.google.com/github/katrin2202/NLP/blob/main/HW_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Работа с объектами-контейнерами и настройками spaCy под свои нужды

##Объекты-контейнеры бибилиотеки spaCy

###Получение индекса токена в объекте Doc

In [63]:
import spacy
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

Здесь вызываем конструктор класса `Doc` и передаем ему два параметра: объект `vocab` — контейнер хранилища со словарными данными и список токенов для добавления в создаваемый объект `Doc`

In [64]:
doc = Doc(Vocab(), words=[u'Hi', u'there'])
doc

Hi there 

### Обход в цикле синтаксических дочерних элементов токена

Нам потребуется установить и загрузать пакет обученного конвейера по умолчанию

In [87]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [88]:
nlp = spacy.load("en_core_web_sm")

Для получения программным образом левосторонних дочерних элементов токена *apple* в данном предложении можно воспользоваться следующим кодом:

In [67]:
doc = nlp("I want a green apple.")
[w for w in doc[4].lefts]

[a, green]

У слова *apple* есть только левосторонние синтаксические дочерние элементы. На практике это означает, что можно заменить атрибут `Token.lefts` на `Token.children`, служащий для поиска всех дочерних элементов токена:

In [68]:
[w for w in doc[4].children]

[a, green]

Можно также использовать атрибут `Token.rights` для получения правосторонних синтаксических дочерних элементов:
Но слово *apply* само является правосторонним дочерним элементом слова want

In [69]:
[w for w in doc[4].rights]

[]

### Контейнер `doc.sents`

Cмысл синтаксических меток, присваиваемых токенам, проявляется лишь в контексте предложения, где встречается данный токен.
С помощью свойства `doc.sents` объекта `Doc` текст можно разделить на отдельные предложения, как показано в следующем примере:

Здесь проходим по предложениям из объекта `doc`, создавая отдельный список токенов для каждого предложения

In [70]:
doc = nlp(u'A severe storm hit the beach. It started to rain.')
for sent in doc.sents:
  print([sent[i] for i in range(len(sent))])

[A, severe, storm, hit, the, beach, .]
[It, started, to, rain, .]


В то же время можно ссылаться на токены в состоящем из множества предложений тексте с помощью глобальных индексов уровня документа, как показано вот здесь:

In [71]:
[doc[i] for i in range(len(doc))]

[A, severe, storm, hit, the, beach, ., It, started, to, rain, .]

Возможность ссылаться на объекты `Token` в документе по их индексам уровня предложения удобна, когда нужно, например, проверить, является ли первое слово во втором предложении обрабатываемого текста местоимением

In [72]:
for i,sent in enumerate(doc.sents):
  if i==1 and sent[0].pos_== 'PRON':
    print('The second sentence begins with a pronoun.')

The second sentence begins with a pronoun.


Выбор первого слова в предложении — элементарная задача, поскольку его индекс всегда равен 0. А как насчет последнего? Например, что делать, если необходимо определить, сколько предложений в тексте оканчивается глаголом (не считая точек и прочих знаков препинания)?

Хотя длины предложений различны, их можно легко вычислить с помощью функции `len()`. Вычитаем 2 из значения `len(sent)` по следующим причинам: во-первых, индексы всегда начинаются с 0 и заканчиваются на *size-1*, во-вторых, последний токен в обоих предложениях нашего примера текста — точка, которую не нужно учитывать.

In [73]:
counter = 0
for sent in doc.sents:
  if sent[len(sent)-2].pos_ == 'VERB':
    counter+=1
print(counter)

1


### Контейнер `doc.noun_chunks`

С помощью свойства `doc.noun_chunks` объекта `Doc` можно пройти по именным фрагментам. Именной фрагмент (*noun chunk*) — это фраза, главным элементом которой является существительное.

In [89]:
doc = nlp(u'A noun chunk is a phrase that has a noun as its head.')
for chunk in doc.noun_chunks:
  print(chunk)

A noun chunk
a phrase
that
a noun
its head


Или же для извлечения именных фрагментов можно обойти в цикле существительные в предложении и найти синтаксические дочерние элементы каждого существительного, чтобы из них образовать именные фрагменты.

In [90]:
for token in doc:
  if token.pos_=='NOUN':
    chunk = ''
    for w in token.children:
       if w.pos_ == 'DET' or w.pos_ == 'ADJ':
         chunk = chunk + w.text + ' '
    chunk = chunk + token.text
    print(chunk)

noun
A chunk
a phrase
a noun
head


##Объект Span

Объект *Span* (от англ. span — «интервал») представляет собой часть объекта `Doc`.

In [117]:
doc=nlp('I want a green apple.')
doc[2:5]

a green apple

Объект Span включает несколько методов, самый интересный из которых — `span.merge()`. С его помощью интервал можно объединять в единый токен, производя повторную токенизацию документа.

In [148]:
doc = nlp(u'The Golden Gate Bridge is an iconic landmark in San Francisco.')
[doc[i] for i in range(len(doc))]

[The, Golden, Gate, Bridge, is, an, iconic, landmark, in, San, Francisco, .]

Каждому слову и знаку препинания соответствует отдельный токен.

С помощью метода `span.merge()` можно изменить поведение по умолчанию:



---

Поправка *Spacy* отказался от этого `span.merge()`метода с тех пор, как был создан этот учебник. Сейчас это можно сделать с помощью `doc.retokenize()`: https://spacy.io/api/doc#retokenize . 

---



In [149]:
with doc.retokenize() as retokenizer:
  attrs = {"LEMMA": "Golden Gate Bridge"}
  retokenizer.merge(doc[1:4], attrs=attrs)
with doc.retokenize() as retokenizer:
  attrs = {"LEMMA": "San Francisco"}
  retokenizer.merge(doc[7:9], attrs=attrs)
[doc[i] for i in range(len(doc))]

[The, Golden Gate Bridge, is, an, iconic, landmark, in, San Francisco, .]

In [150]:
for token in doc:
  print("{:<18}\t{:<18}\t{:<5}\t{:<5}".format(token.text, token.lemma_, token.pos_, token.dep_))

The               	the               	DET  	det  
Golden Gate Bridge	Golden Gate Bridge	PROPN	nsubj
is                	be                	AUX  	ROOT 
an                	an                	DET  	det  
iconic            	iconic            	ADJ  	amod 
landmark          	landmark          	NOUN 	attr 
in                	in                	ADP  	prep 
San Francisco     	San Francisco     	PROPN	pobj 
.                 	.                 	PUNCT	punct


##Настройка конвейера обработки текста под свои нужды

Посмотреть доступные для объекта `nlp` компоненты конвейера можно с помощью команды:

In [151]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

###Отключение компонентов конвейера

In [152]:
nlp = spacy.load('en_core_web_sm', disable=['parser'])

В данном случае мы создадим конвейер обработки без утилиты разбора зависимостей. При вызове такого экземпляра `nlp` для конкретного текста токены в этом тексте не получат метки зависимостей.

In [155]:
doc = nlp(u'I want a green apple.')
for token in doc:
  print("{:<6}\t{:<6}\t{:<5}".format(token.text, token.pos_, token.dep_))

I     	PRON  	     
want  	VERB  	     
a     	DET   	     
green 	ADJ   	     
apple 	NOUN  	     
.     	PUNCT 	     


###Пошаговая загрузка модели

In [194]:
nlp = spacy.load('en_core_web_sm')

Можете выяснить, где именно модель находится в вашей системе. Поможет вспомогательная функция `get_package_path`:

In [157]:
from spacy import util
util.get_package_path('en_core_web_sm')

PosixPath('/usr/local/lib/python3.7/dist-packages/en_core_web_sm')

Посмотрим модель и версию

In [158]:
print(nlp.meta['lang'] + '_' + nlp.meta['name'] + '-' + nlp.meta['version'])

en_core_web_sm-3.4.0


Иногда полезно взглянуть на список компонентов конвейера, используемых с моделью. Список можно получить из поля `pipeline` атрибута `nlp.meta`

In [159]:
nlp.meta['pipeline']

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

*код не работает*

In [ ]:
lang = 'en'
pipeline = ['tagger', 'parser', 'ner']
model_data_path = '/usr/local/lib/python3.7/dist-packages/en_core_web_sm-3.4.0'
lang_cls = spacy.util.get_lang_class(lang)
nlp = lang_cls()
for name in pipeline:
  # component = nlp.create_pipe(name)
  nlp.add_pipe(name, name = name)
nlp.from_disk(model_data_path, exclude=pipeline)

###Настройка компонентов конвейера под свои нужды

In [169]:
doc = nlp(u'I need a taxi to Festy.')
for ent in doc.ents:
  print(ent.text, ent.label_)

Festy WORK_OF_ART


Метка `WORK_OF_AR` обозначает произведение искусства. Но нам нужно, чтобы средство распознавания сущностей классифицировало его как сущность типа `DISTRICT`.

Сначала добавим новую метку `DISTRICT` в список поддерживаемых типов сущностей.

In [170]:
LABEL = 'DISTRICT'
TRAIN_DATA = [
    ('We need to deliver it to Festy.', {
        'entities': [(25, 30, 'DISTRICT')]
    }),
    ('I like red oranges', {
        'entities': []
    })
  ]

Следующий этап — добавление новой метки сущности `DISTRICT` в компонент распознавания сущностей.

In [171]:
ner = nlp.get_pipe('ner')

Выполнив этот шаг, в полученный объект `ner` можно добавить новую метку с помощью метода `ner.add_label()`:

In [172]:
ner.add_label(LABEL)

1

Прежде чем приступить к обучению средства распознавания сущностей, необходимо отключить остальные конвейеры, чтобы во время обучения обновлялся только компонент распознавания сущностей:

In [173]:
nlp.disable_pipes('tagger')
nlp.disable_pipes('parser')

['parser']

Теперь можно начинать обучение компонента распознаванию сущностей на примерах данных из списка `TRAIN_DATA`, который был создан ранее в этом разделе:

*код исправлен*

In [182]:
optimizer = nlp.create_optimizer()
import random
from spacy.training.example import Example
for i in range(25):
  random.shuffle(TRAIN_DATA)
  for text, annotations in TRAIN_DATA:
    example = Example.from_dict(doc, annotations)
    nlp.update([example], sgd=optimizer)

По завершении выполнения можно проверить, как обновленный оптимизатор распознает токен Festy:

*не вышло*

In [183]:
doc = nlp(u'I need a taxi to Festy.')
for ent in doc.ents:
  print(ent.text, ent.label_)

Festy WORK_OF_ART


/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Теперь при необходимости можно загрузить обновленный компонент в новом сеансе, используя метод from_disk(). Чтобы убедиться в этом, закройте текущий сеанс интерпретатора, откройте новый и выполните следующий код:


*не работает*

In [ ]:
ner.to_disk('/usr/to/ner')

In [ ]:
import spacy
from spacy.pipeline import EntityRecognizer
nlp = spacy.load('en_core_web_sm', disable=['ner'])
ner = EntityRecognizer(nlp.vocab)
ner.from_disk('/usr/to/ner')
nlp.add_pipe(ner)

In [195]:
doc = nlp(u'We need to deliver it to Festy.')
for ent in doc.ents:
  print(ent.text, ent.label_)

Festy WORK_OF_ART


##Использование структур данных уровня языка С библиотеки spaCy

###Подготовка рабочей среды и получение текстовых файлов

In [196]:
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Сценарий Cypthon

Создаем в одном из каталогов локальной файловой системы файл `spacytext.pyx` и вставляем в него следующий код:

In [209]:
f = open('spacytext.pyx', 'w')

In [210]:
f.write("""from cymem.cymem cimport Pool
from spacy.tokens.doc cimport Doc
from spacy.structs cimport TokenC
from spacy.typedefs cimport hash_t

cdef struct DocStruct:
  TokenC* c
  int length

cdef int counter(DocStruct* doc, hash_t tag):
  cdef int cnt = 0
  for c in doc.c[:doc.length]:
    if c.tag == tag:
      cnt += 1
  return cnt

cpdef main(Doc mydoc):
  cdef int cnt
  cdef Pool mem = Pool()
  cdef DocStruct* doc_ptr = <DocStruct*>mem.alloc(1, sizeof(DocStruct))
  doc_ptr.c = mydoc.c
  doc_ptr.length = mydoc.length
  tag = mydoc.vocab.strings.add('PRP')
  cnt = counter(doc_ptr, tag)
  print(doc_ptr.length)
  print(cnt)""")

623

In [211]:
f.close()

###Сборка модуля Cython

Создаем файл `setup.py` в каталоге, где располагается наш сценарий Cython. Файл должен содержать следующий код:

In [215]:
f = open('setup.py', 'w')

In [216]:
f.write('''from distutils.core import setup
from Cython.Build import cythonize

import numpy
setup(name='spacy text app',
      ext_modules=cythonize("spacytext.pyx", language="c++"),
      include_dirs=[numpy.get_include()]
      )''')

221

In [217]:
f.close()

После подготовки установочного сценария компилируем код Cython.
Сделать это можно из системного терминала:

In [218]:
!python setup.py build_ext --inplace

running build_ext
building 'spacytext' extension
creating build
creating build/temp.linux-x86_64-cpython-37
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c spacytext.cpp -o build/temp.linux-x86_64-cpython-37/spacytext.o
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1969:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from spacytext.cpp:774:
/usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_V

###Тестирование модуля

После успешного завершения процесса компиляции модуль `spacytext` будет добавлен в среду Python. Для его тестирования откройте сеанс Python и выполните команду:

In [219]:
from spacytext import main

In [223]:
import spacy
nlp = spacy.load('en_core_web_sm')
f= open("test.txt","rb")
contents =f.read()
doc = nlp(contents[:100000].decode('utf8'))
main(doc)

73
4


#Выделение и использование лингвистических признаков

###Теги для чисел, символов и знаков препинания

Для начала выделим из токенов признаки общих частей речи и увидим, как spaCy распознает различные части речи:

Мы создали для входного предложения объект Doc и вывели теги общих частей речи, а также воспользовались функцией `spacy.explain()`, которая возвращает описание для заданного лингвистического признака.

In [275]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"The firm earned $1.5 million in 2017.")
for token in doc:
  print("{:10}\t{:10}\t{}".format(token.text, token.pos_, spacy.explain(token.pos_)))

The       	DET       	determiner
firm      	NOUN      	noun
earned    	VERB      	verb
$         	SYM       	symbol
1.5       	NUM       	numeral
million   	NUM       	numeral
in        	ADP       	adposition
2017      	NUM       	numeral
.         	PUNCT     	punctuation


Теперь сравним теги общих и уточненных частей речи для того же предложения, выведя в отдельном столбце описание для тегов уточненных частей речи:

Второй и третий столбцы содержат теги общих и уточненных частей речи соответственно. В четвертом столбце приведено описание тегов уточненных частей речи из третьего столбца.

In [227]:
for token in doc:
  print("{:10}\t{:10}\t{:3}\t{}".format(token.text, token.pos_, token.tag_, spacy.explain(token.tag_)))

The       	DET       	DT 	determiner
firm      	NOUN      	NN 	noun, singular or mass
earned    	VERB      	VBD	verb, past tense
$         	SYM       	$  	symbol, currency
1.5       	NUM       	CD 	cardinal number
million   	NUM       	CD 	cardinal number
in        	ADP       	IN 	conjunction, subordinating or preposition
2017      	NUM       	CD 	cardinal number
.         	PUNCT     	.  	punctuation mark, sentence closer


###Выделение описаний денежных сумм

Следующий сценарий иллюстрирует, как фразу "$1.5 million" можно выделить из предложения на основе одних лишь тегов частей речи токенов. Можете сохранить этот сценарий в файле и затем выполнить код из сеанса Python:

В этом коде проходим по токенам предложения в цикле в поисках токена с тегом $ уточненной части речи. Данный тег обозначает символ валюты, с которого обычно начинается фраза, описывающая некоторую сумму денег.

In [228]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"The firm earned $1.5 million in 2017.")
phrase = ''
for token in doc:
  if token.tag_ == '$':
    phrase = token.text
    i = token.i+1
    while doc[i].tag_ == 'CD':
      phrase += doc[i].text + ' '
      i += 1
    break
phrase = phrase[:-1]
print(phrase)

$1.5 million


###Преобразование утвердительных высказываний в вопросительные

Предложение содержит несколько глаголов и местоимений, причем с различной морфологией. Чтобы в этом убедиться, взглянем на теги частей речи, которые spaCy присваивает токенам предложения:

In [236]:
doc = nlp(u"I can promise it is worth your time.")
for token in doc:
  print("{:7}\t{:5}\t{}".format(token.text, token.pos_, token.tag_))

I      	PRON 	PRP
can    	AUX  	MD
promise	VERB 	VB
it     	PRON 	PRP
is     	AUX  	VBZ
worth  	ADJ  	JJ
your   	PRON 	PRP$
time   	NOUN 	NN
.      	PUNCT	.


Основные шаги генерации вопроса из исходного утверждения.

1. Поменять порядок слов в исходном предложении с «подлежащее + вспомогательный модальный глагол + глагол в неопределенной форме» на «модальный вспомогательный глагол + глагол в неопределенной форме + подлежащее».
2. Заменить личное местоимение I (подлежащее в предложении) на you.
3. Заменить притяжательное местоимение your на my.
4. Вставить наречие-модификатор really перед словом promise для усиления последнего.
5. Заменить знак препинания . на ? в конце предложения.

Эти шаги реализованы в следующем сценарии:

In [247]:
doc = nlp(u"I can promise it is worth your time.")
doc

I can promise it is worth your time.

Прежде всего проходим в цикле по токенам предложения и меняем местами 
существительное и глагол, чтобы предложение стало вопросительным.

In [256]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"I can promise it is worth your time.")
sent = ''
for i,token in enumerate(doc):
  if token.tag_ == 'PRP' and doc[i+1].tag_ == 'MD' and doc[i+2].tag_ == 'VB':
    sent = doc[i+1].text.capitalize() + ' ' + doc[i].text
    sent = sent + ' ' + doc[i+2:].text
    break
sent

'Can I promise it is worth your time.'

Далее создаем новый цикл `for`, который заменит личное местоимение `I` личным местоимением you. Для этого ищем личные местоимения (помеченные тегами `PRP`). Если личное местоимение — `I`, меняем его на `you` и выходим из цикла `for`.

In [258]:
doc=nlp(sent)
for i,token in enumerate(doc):
  if token.tag_ == 'PRP' and token.text == 'I':
    print(doc[:i].text)
    sent = doc[:i].text + ' you ' + doc[i+1:].text
    break
sent

'Can you promise it is worth your time.'

Повторяем этот процесс. Ищем тег `PRP$` и меняем притяжательное местоимение `your` на `my`.

In [259]:
doc=nlp(sent)
for i,token in enumerate(doc):
  if token.tag_ == 'PRP$' and token.text == 'your':
    sent = doc[:i].text + ' my ' + doc[i+1:].text
    break
sent

'Can you promise it is worth my time.'

В новом цикле `for` находим глагол в неопределенной форме и вставляем перед ним наречие-модификатор `really`

In [260]:
doc=nlp(sent)
for i,token in enumerate(doc):
  if token.tag_ == 'VB':
    sent = doc[:i].text + ' really ' + doc[i:].text
    break
sent

'Can you really promise it is worth my time.'

Наконец, заменяем точку в конце предложения вопросительным знаком: это единственный шаг, для которого не нужен цикл.

In [244]:
doc=nlp(sent)
sent = doc[:len(doc)-1].text + '?'
sent

'Can I promise it is worth your time?'

##Использование меток синтаксических зависимостей при обработке языка

###Различаем подлежащие и дополнения

Чтобы определить программным образом, чем в заданном предложении являются такие местоимения, как you или it, необходимо посмотреть на присвоенную им метку зависимости. Теги частей речи в сочетании с метками зависимостей позволяют получить гораздо больше информации о роли токена в предложении.

Второй и третий столбцы содержат теги общих и уточненных частей речи соответственно. Четвертый столбец содержит метки зависимостей, а пятый — описания этих меток.

In [264]:
doc = nlp(u"I can promise it is worth your time.")
for token in doc:
  print("{:5}\t{:5}\t{:5}\t{:10}\t{}".format(token.text, token.pos_, token.tag_, token.dep_, spacy.explain(token.dep_)))

I    	PRON 	PRP  	nsubj     	nominal subject
can  	AUX  	MD   	aux       	auxiliary
promise	VERB 	VB   	ROOT      	root
it   	PRON 	PRP  	nsubj     	nominal subject
is   	AUX  	VBZ  	ccomp     	clausal complement
worth	ADJ  	JJ   	acomp     	adjectival complement
your 	PRON 	PRP$ 	poss      	possession modifier
time 	NOUN 	NN   	npadvmod  	noun phrase as adverbial modifier
.    	PUNCT	.    	punct     	punctuation


###Выясняем, какой вопрос далжен задать чат-бот

Начнем с импорта модуля sys, который позволяет получить предложение в виде аргумента для дальнейшей обработки:

In [288]:
import spacy
import sys
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

Далее опишем функцию для распознавания и извлечения произвольного именного фрагмента — прямого дополнения из входного документа. Например, если вы ввели документ, содержащий предложение *I want a green apple.*, то будет возвращен фрагмент *a green apple*:

In [289]:
def find_chunk(doc):
  chunk = ''
  for i,token in enumerate(doc):
    if token.dep_ == 'dobj':
      shift = len([w for w in token.children])
      #print([w for w in token.children])
      сhunk = doc[i-shift:i+1]
      break
  return chunk

Следующая функция просматривает фрагмент и определяет, какой тип вопроса должен задать чат-бот:

Сначала задаем начальное значение переменной `question_type` равным 'yesno', что соответствует вопросу типа «да/нет». Далее в переданном в функцию `chunk` ищем токен с тегом 'amod' который означает прилагательное-модификатор. Если таковое находится, меняем значение переменной `question_type` на 'info', соответствующее информационному типу вопроса.

In [290]:
def determine_question_type(chunk):
  question_type = 'yesno'
  for token in chunk:
    if token.dep_ == 'amod':
      question_type = 'info'
  return question_type

Определив, какой тип вопроса нам нужен, генерируем в следующей функции вопрос на основе входного предложения:

In [291]:
def generate_question(doc, question_type):
  sent = ''
  for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and doc[i+1].tag_ == 'VBP':
      sent = 'do ' + doc[i].text
      sent = sent + ' ' + doc[i+1:].text
      break
  doc=nlp(sent)
  for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and token.text == 'I':
      sent = doc[:i].text + ' you ' + doc[i+1:].text
      break
  doc=nlp(sent)
  if question_type == 'info':
    for i,token in enumerate(doc):
      if token.dep_ == 'dobj':
        sent = 'why ' + doc[:i].text + ' one ' + doc[i+1:].text
        break
  if question_type == 'yesno':
    for i,token in enumerate(doc):
      if token.dep_ == 'dobj':
        sent = doc[:i-1].text + ' a red ' + doc[i:].text
        break
  doc=nlp(sent)
  sent = doc[0].text.capitalize() +' ' + doc[1:len(doc)-1].text + '?'
  return sent

Обратите внимание: используемый алгоритм предполагает, что входное предложение оканчивается знаком препинания, например . или !.

После описания всех функций посмотрим на основной блок сценария:

In [330]:
def find_chunk(doc):
  chunk = ''
  for i,token in enumerate(doc):
    if token.dep_ == 'dobj':
      shift = len([w for w in token.children])
      chunk = doc[i-shift:i+1]
      break
  # print(chunk)
  return chunk

def determine_question_type(chunk):
  question_type = 'yesno'
  for token in chunk:
    if token.dep_ == 'amod':
      question_type = 'info'
  return question_type

def generate_question(doc, question_type):
  sent = ''
  for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and doc[i+1].tag_ == 'VBP':
      sent = 'do ' + doc[i].text
      sent = sent + ' ' + doc[i+1:].text
      break
  doc=nlp(sent)
  for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and token.text == 'I':
      sent = doc[:i].text + ' you ' + doc[i+1:].text
      break
  doc=nlp(sent)
  if question_type == 'info':
    for i,token in enumerate(doc):
      if token.dep_ == 'dobj':
        sent = 'why ' + doc[:i].text + ' one ' + doc[i+1:].text
        break
  if question_type == 'yesno':
    for i,token in enumerate(doc):
      if token.dep_ == 'dobj':
        sent = doc[:i-1].text + ' a red ' + doc[i:].text
        break
  doc=nlp(sent)
  sent = doc[0].text.capitalize() +' ' + doc[1:len(doc)-1].text + '?'
  return sent

def chat_bot(new_str):
  if len(new_str) > 1:
    sent = new_str
    # print("Sent: " + sent)
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sent)
    # print(f"Doc: {doc}")
    chunk = find_chunk(doc)
    # print(f"Chunk: {chunk}")
    if chunk == None:
      print('The sentence does not contain a direct object.')
      sys.exit()
    question_type = determine_question_type(chunk)
    question = generate_question(doc, question_type)
    print(question)
  else:
    print('You did not submit a sentence!')


s1 = 'I want a green apple.'
print(s1)
chat_bot(s1)
s2 = 'I want an apple.'
print(s2)
chat_bot(s2)
s3 = 'I want...'
print(s3)
chat_bot(s3)
s4 = ""
print(s4)
chat_bot(s4)

I want a green apple.
Why do you want a green one?
I want an apple.
Do you want a red apple?
I want...
Do you want?

You did not submit a sentence!
